# Modelo Naive Bayes (Multinomial y Gaussiano)

En el presente notebook se construyen dos modelos Multinomial y Gaussiano a partir del conjunto de datos que sigue 
la estructura de la tabla mostrada abajo. Para esta tabla **blue**, **green**, **red**, **nir**, **swir1** y 
**swir2** son las características de los pixeles de una escena landsat y **wofs**, **bosque**, **wofs_bosque**,  
**ninguno**, **cloud** son las clases a la cual pertence casa pixel. Un pixel solo pertenece a una única clase.


| blue | green | red | nir | swir1 | swir2 | wofs | bosque | wofs_bosque | ninguno  | cloud |
|------|-------|-----|-----|-------|-------|------|--------|-------------|----------|-------|
|      |       |     |     |       |       |      |        |             |          |       |


* El modelo generado para Multinomial es guardado en el archivo **model_nb_multinomial.joblib** 
* El modelo generado para Gaussiano es guardado en el archivo **model_nb_gaussiano.joblib** 

In [1]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
#import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


/home/datacube/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Se importa el Dataset con informacion de nubes
Como se ha descrito, las imagenes descargadas de los portales de imagenes satelitales presentan información en multiples bandas. Es con esta información que los algoritmos de clasificación pueden emitir un juicio para determinar a que tipo de información puede corresponder un pixel: bosque, no busque,nubes,agua, agua_bosque,ninguno.
En este escenario se plantea probar el clasificador Naive Bayes con información de imagenes que contengan nubes y agregar una nueva clase: Nubes. 
Este escenario se plantea en respuesta a que la clasificación de imagenes satelitales cuyas nubes fueron removidas resultaban un problema facil de clasificar para este tipo de modelo. 

In [2]:
satellite = pd.read_csv('https://docs.google.com/uc?export=download&id=1zIf10M5nkLdneCYU1Kw_KyEOynNp3hb2')
satellite.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,cloud
0,8452.0,8750.0,8900.0,9146.0,6759.0,4585.0,0,0,0,0,1
1,299.0,203.0,100.0,73.0,79.0,60.0,1,0,0,0,0
2,5167.0,4956.0,4945.0,5647.0,3920.0,2588.0,0,0,0,0,1
3,245.0,177.0,93.0,75.0,100.0,80.0,1,0,0,0,0
4,259.0,179.0,92.0,64.0,89.0,72.0,1,0,0,0,0


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo porque el clasificador multinomial Naive bayes no funciona bien con esta información.

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como parte de preparación de los datos para crear el modelo se deben hacer algunos ajustes a la forma en que el dataset muestra la infiromación.Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [5]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [6]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [7]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [8]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

In [9]:
# Reemplazar el valor de columna ninguno por NaN(0) y cloud(1)
satellite['cloud'] = satellite['cloud'].replace(0,np.nan)
satellite['cloud'] = satellite['cloud'].replace(1,'cloud')

# Nueva columna de clasificación
Como ultima parte para la preparación de los datos debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [10]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,cloud,label
0,8452.0,8750.0,8900.0,9146.0,6759.0,4585.0,NaN,NaN,NaN,NaN,cloud,cloud
1,299.0,203.0,100.0,73.0,79.0,60.0,wofs,NaN,NaN,NaN,NaN,wofs
2,5167.0,4956.0,4945.0,5647.0,3920.0,2588.0,NaN,NaN,NaN,NaN,cloud,cloud
3,245.0,177.0,93.0,75.0,100.0,80.0,wofs,NaN,NaN,NaN,NaN,wofs
4,259.0,179.0,92.0,64.0,89.0,72.0,wofs,NaN,NaN,NaN,NaN,wofs
5,1488.0,1590.0,1454.0,3237.0,2131.0,1409.0,NaN,NaN,NaN,NaN,cloud,cloud
6,148.0,315.0,226.0,2556.0,1159.0,507.0,NaN,bosque,NaN,NaN,NaN,bosque
7,2567.0,2692.0,2617.0,4737.0,3540.0,2621.0,NaN,NaN,NaN,NaN,cloud,cloud
8,214.0,386.0,224.0,3028.0,1363.0,564.0,NaN,bosque,NaN,NaN,NaN,bosque
9,351.0,204.0,143.0,178.0,160.0,139.0,wofs,NaN,NaN,NaN,NaN,wofs


wofs
cloud
ninguno
boque
wofs_bosque

In [11]:
# Se crea el conjunto de clasificacion y el de entrenamiento
Y = satellite["label"]
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
Y.value_counts()

cloud          396285
wofs           340457
bosque         214883
ninguno         45196
wofs_bosque       167
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [12]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [13]:
X_test

,blue,green,red,nir,swir1,swir2
915922,259.0,175.0,81.0,61.0,83.0,68.0
176556,5675.0,5509.0,5560.0,5559.0,4377.0,3643.0
333568,2087.0,2062.0,1932.0,3399.0,2331.0,1652.0
655525,723.0,715.0,626.0,691.0,669.0,630.0
270124,197.0,443.0,229.0,3750.0,1564.0,597.0
...,...,...,...,...,...,...
845366,184.0,455.0,256.0,3583.0,1519.0,565.0
843822,194.0,153.0,67.0,55.0,79.0,61.0
476169,693.0,861.0,699.0,3076.0,1629.0,796.0
837933,257.0,556.0,329.0,3798.0,1766.0,732.0


In [14]:
y_test

915922       wofs
176556      cloud
333568      cloud
655525      cloud
270124     bosque
           ...   
845366     bosque
843822       wofs
476169    ninguno
837933     bosque
42694       cloud
Name: label, Length: 299097, dtype: object

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosque y ninguno. 
En esta parte de creación del modelo se evaluaran dos variaciones del clasificador Naive Bayes: Gausiano y multinomial. En el analisis de resultados se contrastaran los resultados obtenidos para cada uno de ellos.

### Multinomial

In [15]:
#Create a Gaussian Classifier
gnb = MultinomialNB()

In [16]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [18]:
Counter(y_pred)

Counter({'wofs': 100042,
         'cloud': 109578,
         'bosque': 59691,
         'ninguno': 27763,
         'wofs_bosque': 2023})

In [19]:
#wofs
#cloud
#ninguno
#bosque
#wofs_bosque
confusion_matrix(y_test, y_pred)

array([[ 58848,      6,   5576,      2,     16],
       [     9, 105766,  11260,     90,   1370],
       [   834,   1532,  10899,    100,    355],
       [     0,   2268,     21,  99848,    254],
       [     0,      6,      7,      2,     28]])

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.91      0.95     64448
       cloud       0.97      0.89      0.93    118495
     ninguno       0.39      0.79      0.53     13720
        wofs       1.00      0.98      0.99    102391
 wofs_bosque       0.01      0.65      0.03        43

    accuracy                           0.92    299097
   macro avg       0.67      0.85      0.68    299097
weighted avg       0.95      0.92      0.93    299097



In [21]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9207347449155291


In [22]:
joblib.dump(gnb, 'model_nb_multinomial.joblib') 

['model_nb_multinomial.joblib']

### Gaussiano

In [23]:
#Create a Gaussian Classifier
gnb = GaussianNB()

In [24]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [26]:
Counter(y_pred)

Counter({'wofs': 98820,
         'cloud': 102322,
         'ninguno': 30141,
         'bosque': 63233,
         'wofs_bosque': 4581})

In [27]:
#wofs
#cloud
#ninguno
#bosque
#wofs_bosque
confusion_matrix(y_test, y_pred)

array([[ 59341,     51,   3229,     58,   1769],
       [    48, 101716,  16719,      0,     12],
       [  3804,    555,   8797,     31,    533],
       [    36,      0,   1392,  98721,   2242],
       [     4,      0,      4,     10,     25]])

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.94      0.92      0.93     64448
       cloud       0.99      0.86      0.92    118495
     ninguno       0.29      0.64      0.40     13720
        wofs       1.00      0.96      0.98    102391
 wofs_bosque       0.01      0.58      0.01        43

    accuracy                           0.90    299097
   macro avg       0.65      0.79      0.65    299097
weighted avg       0.95      0.90      0.92    299097



In [29]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8980364229664624


In [30]:
joblib.dump(gnb, 'model_nb_gaussiano.joblib') 

['model_nb_gaussiano.joblib']

## Analisis de resultados y conclusiones

El rendmiento obtenido con el modelo Multinomial llega a ser del 79.73% y el rendimiento con el modelo Gausiano alcanza el 87.11%. Por lo que logramos determinar que en este escenario funciona mejor el modelo Gassiano que el Multinomial por aproimadamente 8 puntos de diferencia. Revisando la matriz de confusión logramos evidenciar que el modelo Multinomial confunde mucho de los elementos de la clase 3(Ninguno) siendo esto logico porque los valores de caracteristicas de entrada.

**Multinomial**

**Accuracy:** 79.73%

**Matriz de predicción**

| Clase\Predicción | WOFS  | Cloud  | Ninguno | Bosque | WOFS_Bosque |
|------------------|-------|--------|---------|--------|-------------|
| WOFS             | 28721 | 1      | 320     | 2      | 551         |
| Cloud            | 43    | 120350 | 39851   | 240    | 1295        |
| Ninguno          | 1895  | 8902   | 4897    | 1003   | 1646        |
| Bosque           | 0     | 3826   | 175     | 82221  | 278         |
| WOFS_Bosque      | 0     | 12     | 7       | 9      | 50          |
       
**Gaussiano**

**Accuracy:** 87.11%

**Matriz de predicción**

| Clase\Predicción | WOFS  | Cloud  | Ninguno | Bosque | WOFS_Bosque |
|------------------|-------|--------|---------|--------|-------------|
| WOFS             | 27133 | 74     | 1321    | 349    | 718         |
| Cloud            | 68    | 134215 | 27495   | 1      | 0           |
| Ninguno          | 2570  | 1176   | 13427   | 1065   | 105         |
| Bosque           | 95    | 0      | 2685    | 8335   | 385         |
| WOFS_Bosque      | 24    | 0      | 6       | 40     | 8           |

En el Modelo multinomial las clases ninguno y wofs_bosque tienen muy bajas estadisticas de presicion y recall por las siguentes razones: una razón de forma general para obtener tan bajas estadísticas usando el modelo multinomiál es que el trabaja mejor con datos descretos, es decir, valores que esten en un rango de predefinido, por el contrario los datos de características de nuestro dataset se encuentran en valores continuos.
En el modelo Gaussiano obtuvimos mejores características por la razón mencionada anteriormente. Sin embargo logramos apreciar que hay 2 clases(ninguno y wofs_bosque) que no estan teniendo buenas estadísticas de predicción(Precision y recall). Las razones a las que podemos llegar para esto son las siguientes: la clase wofs_bosque(Clase donde se categorizan los pixeles que se encuentran en un rango de valores que se pueden clasificar como Bosque y Agua) es una clase que se encuentra desbalanceada comparada con la cantidad de datos que otras clases tienen. Sin embargo como esta clase no es lo suficientemente determinante para nuestro ejercicio de clasificación decidimos dejarla de esta manera.
Por otro lado la clasificación “ninguno” puede estar presentando un bajo recall comparado con sus demas clases porque es una clasificación donde pueden estar una gran diversidad de valores haciendo que este modelo no generalice bien para esta clase particular.

# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra